In [1]:
import torch
from torch.utils.data import Dataset
import torchvision.transforms as transforms



import numpy as np
import os
path='/mnt/nvme/node02/pranav/AE24/data/split_data'

In [2]:
eeg_file_paths = [os.path.join(path,file) for  file in os.listdir(path) if "eeg" in file and "train" in file]
len(eeg_file_paths)


666

In [ ]:
np.memmap(efp,)

In [19]:
awwe=np.load(efp)

In [20]:
data=awwe[:128,:]

In [27]:
num_samples=128
frame_length=64
hop_length =32
num_frames =1 + (num_samples - frame_length) // hop_length




In [28]:
[print([start,start + frame_length] )for start in range(0, len(data) - frame_length + 1, hop_length)]

[0, 64]
[32, 96]
[64, 128]


[None, None, None]

In [14]:
class maeeg_dataset(Dataset):
    def __init__(self, path='/mnt/nvme/node02/pranav/AE24/data/split_data', type="train",frame_length=64,hop_length=30,):
        
        super(maeeg_dataset, self).__init__()
        self.frame_length=frame_length
        self.hop_length=hop_length

        self.input_paths = [os.path.join(path,file) for  file in os.listdir(path) if "eeg" in file and type in file]
        assert len(self.input_paths) != 0, 'No data found'


        self.index_map = {}
        global_index = 0
        for file_name in self.input_paths:
            data_shape = np.load(file_name, mmap_mode='r').shape
            num_samples = data_shape[0] 

            num_windows = 1 + (num_samples - self.frame_length) // self.hop_length

            for window_offset in range(num_windows):
                self.index_map[global_index] = (file_name, window_offset)
                global_index += 1
        


    def __len__(self):
        return len(self.index_map)
    
    def __getitem__(self,idx):
        file_name, window_offset=self.index_map[idx]
        start=window_offset*self.hop_length
        end=start+self.frame_length
        sample=np.load(file_name,mmap_mode='r')[start:end]

        return torch.from_numpy(sample.copy())




In [16]:
md=maeeg_dataset()
md.__getitem__(14).shape

torch.Size([64, 64])

In [20]:

# DataLoader's batch_size should be 1
data_loader = torch.utils.data.DataLoader(md, batch_size=2400)
